In [ ]:
%pip install "torch==2.4.0"
%pip install "transformers==4.45.1" "datasets==3.0.1" "accelerate==0.34.2" "trl==0.11.1" "peft==0.13.0"

In [3]:
!pip install datasets transformers qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.1 MB/s eta 0:00:00


In [15]:
import io
import json
from PIL import Image
from datasets import load_dataset
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, Qwen2VLProcessor
#from trl import SFTConfig, SFTTrainer
#from peft import LoraConfig
from qwen_vl_utils import process_vision_info



In [ ]:
import wandb
wandb.init(mode="disabled")

In [29]:
# 시스템(assistant)에게 주어진 역할
system_message = "당신은 이미지로부터 문자와 숫자를 판독하는 전문적인 OCR 모델입니다"

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- image: [image]

위 이미지에서 모든 문자,숫자를 판독해서 텍스트로 결과를 출력하세요.
출력 시 **아래 TEXT 형태**를 반드시 지키세요:
result
원재료명 설탕,물엿,젤라틴,D-소르비톨(감미료),산도조절제I,산도조절제II,산도조절제III,혼합제제I(젓산,젓산칼슘,이산화규소),향료3종,당류가공품,펙틴,산도조절제IV,블랙캐롯농축액,혼합재제II(홍화황색소,치자청색소,주정),천연향료(콜라향)
복숭항,밀,우유,대두,땅콩,달걀,토마토 혼입 가능
복숭항,밀,우유,대두,땅콩,달걀,토마토 와 같은 제조시설에서 제조
"""

In [30]:
import io
from PIL import Image
from IPython.display import display
import base64

def load_and_display_image(file_path):
  """
  로컬 JPG 파일을 불러와서 코랩 출력창에 표시합니다.

  Args:
    file_path (str): 읽어올 JPG 파일의 경로.
  """
  try:
    with open(file_path, "rb") as f:
      image_data = f.read()

    # 이미지 데이터를 메모리 버퍼에 씁니다.
    buffer = io.BytesIO(image_data)

    # 버퍼의 시작점으로 돌아갑니다.
    buffer.seek(0)

    # 버퍼에서 이미지를 엽니다.
    image = Image.open(buffer)

      # 이미지를 Base64 문자열로 인코딩
    buffer_for_b64 = io.BytesIO()
    image.save(buffer_for_b64, format='PNG')
    encoded_string = base64.b64encode(buffer_for_b64.getvalue()).decode('utf-8')

    # IPython.display.display 함수를 사용하여 이미지를 출력합니다.
    print("이미지를 출력합니다=",len(encoded_string))
    #display(image)

    return image, encoded_string

  except FileNotFoundError:
    print(f"Error: 파일 '{file_path}'을(를) 찾을 수 없습니다.")
  except Exception as e:
    print(f"Error: 이미지를 불러오는 중 오류가 발생했습니다: {e}")

# 사용 예시:
# 이 코드를 코랩에 붙여넣기 전, 먼저 이미지를 코랩에 업로드해야 합니다.
# 1. 왼쪽 폴더 아이콘 클릭
# 2. '파일 업로드' 버튼 클릭 후 이미지 파일(예: my_photo.jpg) 선택
# 3. 아래 'my_photo.jpg'를 업로드한 파일 이름으로 변경
image_path = './김광무_100.jpg'
image, encoded_string = load_and_display_image(image_path)

이미지를 출력합니다= 1321652


In [31]:

def format_data(image_path):
   #image_path = '/content/김광무_100.jpg'
   image, encoded_string = load_and_display_image(image_path)

   return {
       "messages": [
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": system_message
                   }
               ],
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "image",
                       "image": image_path,
                   },
               ],
           },
           {
               "role": "assistant",
               "content": [
                   {
                       "type": "text",
                       "text": "",
                   }
               ],
           },
       ],
   }

In [50]:
train_list= []

#for x in range(len(train_df)):
train_list.append(format_data('./김광무_112.jpg'))




이미지를 출력합니다= 13912572


In [51]:
print(len(train_list), train_list[0])

1 {'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지로부터 문자와 숫자를 판독하는 전문적인 OCR 모델입니다'}]}, {'role': 'user', 'content': [{'type': 'image', 'image': './김광무_112.jpg'}]}, {'role': 'assistant', 'content': [{'type': 'text', 'text': ''}]}]}


In [41]:
model_id = "Qwen/Qwen2-VL-7B-Instruct"

model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)

processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [52]:
sample_data = train_list[0]['messages']

sample_prompt = processor.apply_chat_template(
    sample_data,
    tokenize=False,
    add_generation_prompt=False,
)

In [53]:
def split_input_and_label(prompt):
    input = prompt.split('<|im_start|>assistant')[0] + '<|im_start|>assistant'
    label = prompt.split('<|im_start|>assistant')[1]
    return input, label

In [54]:
sample_prompt_input, sample_prompt_label = split_input_and_label(sample_prompt)

In [55]:
sample_data_image_inputs, sample_data_video_inputs = process_vision_info(sample_data)

In [56]:
inputs = processor(
    text=[sample_prompt_input],
    images=sample_data_image_inputs,
    videos=sample_data_video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

In [57]:
%%time
# Inference: Generation of the output
generated_ids = model.generate(
    **inputs,
    max_new_tokens=2048,
    temperature=0.1,
    top_p=0.001,
    repetition_penalty=1.05,
)

generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
output_text = processor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
print("추론결과=\n")
print(output_text)


추론결과=

["\n\n이 이미지에는 다음과 같은 정보가 포함되어 있습니다:\n\n1. **제품명**: 배추국수\n2. **원재료**: 면/소맥분(밀:미국산, 호주산), 팥유(말레이시아산), 감자전분(덴마크산), 정제소금, 면류점가알칼리제(탄산칼륨, 탄산나트륨, 세이인산나트륨), 혼합제제{글리세린, d- 토포페롤(혼합형), 폴리글리세린지방산에스테르, 효소분해 레시틴}, 혼합제제(메타인산나트륨, 폴리인산나트륨, 제일 인산나트륨, 피로인산나트륨). 스프루/고추장, 설탕, 기타 과당, 냉동홍고추페이스트, 채종유, 정제수, 소고기맛간장, 양파, 다진양념베이스, 진육수추출액, 정제소금, 화이트식초, 배퓨레, 동치미농축액, 구연산, 참기름, 변성전분, 마늘, 파프 리카추출색소1, 파프리카추출색소2, 혼합제제(덱스트린, 카라멜색소III), 5'-리보뉴클레오티드이나트륨, 호박산이 나트륨, 고소한풍미유, 복음참깨, 복음흑깨, 구운김후레이크.\n3. **밀, 대두, 우유, 쇠고기 함유**\n4. **품목 번호**: 19860360007-405(안성), 보고번호 19720154001-461(부산)\n5. **영양소 비교 표시**: 1350mg (1일 섭취 기준량)\n\n이 정보는 제품의 원재료, 함유물, 영양소 등에 대한 설명을 제공합니다."]
